In [1]:
import ee
import geemap
import geopandas as gpd

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:
Map = geemap.Map()

In [4]:
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2021_REL/TCC/v2021-4')
# Filter collection to 2021 and CONUS study area
tcc = dataset.filter(ee.Filter.calendarRange(2021, 2021, 'year')) \
             .filter(ee.Filter.eq('study_area', 'CONUS')) \
             .first()

In [5]:
# TCC palette
tcc_palette = [
    'CDA066', 'D7C29E', 'C2D096', 'B7D692', 'ADDD8E',
    '78C679', '5CB86B', '41AB5D', '39A156', '329750',
    '238443', '11763D', '006837', '004529'
]

# SE palette
se_palette = ['000000', 'FFFFFF']

In [6]:
# Add layers to the map
Map.addLayer(tcc.select('data_mask'), {'min': 0, 'max': 2}, 'Data Mask', False)
# Map.addLayer(tcc.select('Science_Percent_Tree_Canopy_Cover'), {'min': 0, 'max': 60, 'palette': tcc_palette}, 'Science Percent Tree Canopy Cover')
# Map.addLayer(tcc.select('Science_Percent_Tree_Canopy_Cover_Standard_Error'), {'min': 0, 'max': 4000, 'palette': se_palette}, 'Science Percent Tree Canopy Cover Standard Error')
Map.addLayer(tcc.select('NLCD_Percent_Tree_Canopy_Cover'), {'min': 0, 'max': 60, 'palette': tcc_palette}, 'NLCD Percent Tree Canopy Cover')

In [7]:
Map = geemap.Map()

dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2021_REL/TCC/v2021-4')
# Filter collection to 2021 and CONUS study area
tcc = dataset.filter(ee.Filter.calendarRange(2021, 2021, 'year')) \
             .filter(ee.Filter.eq('study_area', 'CONUS')) \
             .first()

# TCC palette
tcc_palette = [
    'CDA066', 'D7C29E', 'C2D096', 'B7D692', 'ADDD8E',
    '78C679', '5CB86B', '41AB5D', '39A156', '329750',
    '238443', '11763D', '006837', '004529'
]

# SE palette
se_palette = ['000000', 'FFFFFF']

# Add layers to the map
Map.addLayer(tcc.select('data_mask'), {'min': 0, 'max': 2}, 'Data Mask', False)
# Map.addLayer(tcc.select('Science_Percent_Tree_Canopy_Cover'), {'min': 0, 'max': 60, 'palette': tcc_palette}, 'Science Percent Tree Canopy Cover')
# Map.addLayer(tcc.select('Science_Percent_Tree_Canopy_Cover_Standard_Error'), {'min': 0, 'max': 4000, 'palette': se_palette}, 'Science Percent Tree Canopy Cover Standard Error')
Map.addLayer(tcc.select('NLCD_Percent_Tree_Canopy_Cover'), {'min': 0, 'max': 60, 'palette': tcc_palette}, 'NLCD Percent Tree Canopy Cover')

# 1. World Settlement Footprint (WSF) 2015
wsf_dataset = ee.Image('DLR/WSF/WSF2015/v1')
wsf_vis = {'min': 0, 'max': 255}
Map.addLayer(wsf_dataset, wsf_vis, 'Human settlement areas')

# 2. Black Background Layer
opacity = 0.75
black_background = ee.Image(0)
Map.addLayer(black_background, {}, 'Black background', True, opacity)

# 3. Population Density (CIESIN/GPWv411/GPW_Population_Density)
population_dataset = ee.ImageCollection('CIESIN/GPWv411/GPW_Population_Density').first()
population_raster = population_dataset.select('population_density')
population_vis = {
    'max': 1000.0,
    'palette': ['ffffe7', 'FFc869', 'ffac1d', 'e17735', 'f2552c', '9f0c21'],
    'min': 200.0
}
Map.addLayer(population_raster, population_vis, 'population_density')

# 4. Global Tidal Wetland Change (JCU/Murray/GIC/global_tidal_wetland_change/2019)
tidal_dataset = ee.Image('JCU/Murray/GIC/global_tidal_wetland_change/2019')

# Visualization settings for different layers in the dataset
plasma = ['0d0887', '3d049b', '6903a5', '8d0fa1', 'ae2891', 'cb4679', 'df6363', 'f0844c', 'faa638', 'fbcc27', 'f0f921']
loss_palette = ['fe4a49']
gain_palette = ['2ab7ca']
viridis = ['440154', '414487', '2a788e', '22a884', '7ad151', 'fde725']
class_palette = ['9e9d9d', 'ededed', 'ff9900', '009966', '960000', '006699']

# Adding each layer with its specific visualization
# Map.addLayer(tidal_dataset.select('twprobabilityStart'), {'palette': plasma, 'min': 0, 'max': 100}, 'twprobabilityStart', False, 1)
# Map.addLayer(tidal_dataset.select('twprobabilityEnd'), {'palette': plasma, 'min': 0, 'max': 100}, 'twprobabilityEnd', False, 1)
Map.addLayer(tidal_dataset.select('loss'), {'palette': loss_palette, 'min': 1, 'max': 1}, 'Tidal wetland loss', True, 1)
Map.addLayer(tidal_dataset.select('gain'), {'palette': gain_palette, 'min': 1, 'max': 1}, 'Tidal wetland gain', True, 1)
# Map.addLayer(tidal_dataset.select('lossYear'), {'palette': viridis, 'min': 4, 'max': 19}, 'Year of loss', False, 0.9)
# Map.addLayer(tidal_dataset.select('gainYear'), {'palette': viridis, 'min': 4, 'max': 19}, 'Year of gain', False, 0.9)
# Map.addLayer(tidal_dataset.select('lossType'), {'palette': class_palette, 'min': 0, 'max': 5}, 'Loss type', False, 0.9)
# Map.addLayer(tidal_dataset.select('gainType'), {'palette': class_palette, 'min': 0, 'max': 5}, 'Gain type', False, 0.9)

# Bio-Intactness Dataset
bdi_ic = ee.ImageCollection("projects/ebx-data/assets/earthblox/IO/BIOINTACT")
bdi_2017_20_comp = bdi_ic.mean()
bdi_2017_comp = bdi_ic.filterDate('2017-01-01', '2017-12-31').mean()

# Visualization for Bio-Intactness
bdi_vis = {
    'bands': ['BioIntactness'],
    'min': 0,
    'max': 1,
    'palette': ['e5f5e0', 'a1d99b', '31a354']
}

# Add Bio-Intactness layers
Map.addLayer(bdi_2017_20_comp, bdi_vis, "composite 2017-20")
Map.addLayer(bdi_2017_comp, bdi_vis, "composite 2017")

# Biodiversity Datasets
biodiv_biome = ee.Image("projects/sat-io/open-datasets/naturemap/biodiversity/minshort_speciestargets_biome_esh10km_repruns10_ranked")
biodivcarbon = ee.Image("projects/sat-io/open-datasets/naturemap/biodiversity-carbon/minshort_speciestargets_carbon_esh10km_repruns10_ranked")
biodivcarbonwater = ee.Image("projects/sat-io/open-datasets/naturemap/biodiversity-carbon-water/minshort_speciestargets_carbon_water_esh10km_repruns10_ranked")

# Visualization for Biodiversity and Carbon
biodiv_carbon_vis = {
    'min': 1, 
    'max': 100, 
    'palette': ['7a0403', 'c92903', 'f56918', 'fbb938', 'c9ef34', '74fe5d', '1be5b5', '35abf8', '4662d8', '30123b']
}

# Add Biodiversity and Carbon layers
Map.addLayer(biodivcarbon, biodiv_carbon_vis, "Biodiversity and Carbon")
Map.addLayer(biodivcarbonwater, biodiv_carbon_vis, "Biodiversity, Carbon, and Water")

# Top 10% Value for Biodiversity Only
bio30x30 = biodivcarbon.expression("b(0) <= 10")
Map.addLayer(bio30x30.mask(bio30x30.eq(1)), {'palette': ['red']}, "Top 10% value for biodiversity only", False)

dataset = ee.Image('UMD/hansen/global_forest_change_2022_v1_10')
treeCoverVisParam = {
  'bands': ['treecover2000'],
  'min': 0,
  'max': 100,
  'palette': ['black', 'green']
}
Map.addLayer(dataset, treeCoverVisParam, 'tree cover')

treeLossVisParam = {
  'bands': ['lossyear'],
  'min': 0,
  'max': 22,
  'palette': ['yellow', 'red']
}
Map.addLayer(dataset, treeLossVisParam, 'tree loss year')


# Set the map center to New Jersey with an appropriate zoom level
Map.setCenter(-74.4057, 40.0583, 7)  # Longitude, Latitude, Zoom Level

# Show the map
Map

Map(center=[40.0583, -74.4057], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [13]:
import pygris

In [15]:
skylands = gpd.read_file("C:\\Users\\Nissim\\Downloads\\conversation_priority_areas_skylands.geojson")

In [16]:
skylands.head()

,OBJECTID,SITECODE,SITENAME,VERSION,COUNTY,QUADNAME,MUNICIPALI,DESCRIPTIO,BOUNDJUST,BIODIVRANK,SITECLASS,BIODIVCOMM,Shape_Length,Shape_Area,geometry
0,1,S.USNJHP1*148,Burnt Mills,2007_03,Somerset,Raritan ; Gladstone,Bedminster Twp ; Branchburg Twp,The site contains a large mature floodplain fo...,Primary bounds drawn to include mature floodpl...,B4V5,Standard Site,The site contains a good quality occurrence of...,16126.229345,8.952363e+06,"POLYGON ((-74.68042 40.62887, -74.68042 40.628..."
1,2,S.USNJHP1*149,Buttermilk Bridge Site,2007_03,Warren,Bloomsbury,Franklin Twp ; Washington Twp,Wooded slope adjacent to railroad.,Includes extent of upland habitat and railroad...,B5V1,Standard Site,Only known location of a state listed endanger...,42944.202966,2.023485e+07,"POLYGON ((-75.01319 40.73544, -75.01318 40.735..."
2,3,S.USNJHP1*100,Baseball Swamp Site,2007_03,Cape May,Stone Harbor,Middle Twp,The site is on the outer coastal plain of Cape...,The primary bounds include the headwater wetla...,B4,Standard Site,"An occurrence of a state endangered, federally...",10737.013318,5.738767e+06,"POLYGON ((-74.82519 39.10720, -74.82515 39.107..."
3,4,S.USNJHP1*56,Batsto Macrosite,2007_03,Burlington ; Atlantic ; Camden,Atsion ; Indian Mills ; Jenkins ; Hammonton ; ...,Washington Twp ; Shamong Twp ; Hammonton Town ...,The site contains numerous pine barren savanna...,Boundaries were drawn to include undeveloped p...,B1,Macrosite,Numerous exemplary globally rare communities a...,233665.168678,2.025442e+09,"POLYGON ((-74.63950 39.62514, -74.63949 39.625..."
4,5,S.USNJHP1*104,Beagle Club Woods,2007_03,Camden,Clementon,Voorhees Twp,Extensive wooded swamp with spring seepage are...,Primary boundary includes extent of wetland ha...,B4,Standard Site,Occurrence of a federally listed threatened pl...,18292.030709,2.318913e+07,"POLYGON ((-74.94452 39.84912, -74.94304 39.846..."


In [9]:
import geemap.gpd_to_ee
nj_ee_geometry = geemap.gpd_to_ee(states)

ModuleNotFoundError: No module named 'geemap.gpd_to_ee'

In [ ]:
tcc_clipped = tcc.clip(nj_ee_geometry)

In [ ]:
task = ee.batch.Export.image.toDrive(**{
  'image': tcc_clipped,
  'description': 'tcc_clipped_to_NJ',
  'folder': 'ExportedImages',
  'scale': 30,  # Specify appropriate scale for your dataset
  'region': nj_ee_geometry,
  'fileFormat': 'GeoTIFF'
})
task.start()